In [58]:











.+++++++++++++++++++++
.............................................................................................................................................................





















# Import necessary libraries
import pandas as pd
import re
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost as xgb

In [59]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ANKUSH\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [60]:
Train = pd.read_csv(r"C:\Users\ANKUSH\Downloads\NLP-with-Disaster-Tweets-main\NLP-with-Disaster-Tweets-main\train.csv")
Train.drop(['id', 'keyword', 'location'], inplace=True, axis=1)

In [61]:
print(Train)

                                                   text  target
0     Our Deeds are the Reason of this #earthquake M...       1
1                Forest fire near La Ronge Sask. Canada       1
2     All residents asked to 'shelter in place' are ...       1
3     13,000 people receive #wildfires evacuation or...       1
4     Just got sent this photo from Ruby #Alaska as ...       1
...                                                 ...     ...
7608  Two giant cranes holding a bridge collapse int...       1
7609  @aria_ahrary @TheTawniest The out of control w...       1
7610  M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...       1
7611  Police investigating after an e-bike collided ...       1
7612  The Latest: More Homes Razed by Northern Calif...       1

[7613 rows x 2 columns]


In [62]:
Train['text'] = Train['text'].str.lower()

In [63]:
for i in range(len(Train)):
    Train['text'][i] = re.sub(r'http\S+', '', Train['text'][i])

C:\Users\ANKUSH\AppData\Local\Temp\ipykernel_14404\2749212976.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Train['text'][i] = re.sub(r'http\S+', '', Train['text'][i])


In [64]:
Punc = list('''!()-[]{};:'"\,<>./?@#$%^&*_~''')
for p in Punc:
    Train['text'] = Train['text'].str.replace(p, '',regex = True)

In [65]:
Tknzr = TweetTokenizer()
Train['text'] = Train['text'].apply(lambda x: Tknzr.tokenize(x))

In [66]:
Lem = WordNetLemmatizer()
Train['text'] = Train['text'].apply(lambda tokens: [Lem.lemmatize(word) for word in tokens])

In [67]:
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ANKUSH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [68]:
st = stopwords.words('english')
for i in range(len(Train)):
    Train['text'][i] = " ".join([x for x in Train['text'][i] if x not in st])

C:\Users\ANKUSH\AppData\Local\Temp\ipykernel_14404\3266534853.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Train['text'][i] = " ".join([x for x in Train['text'][i] if x not in st])


In [75]:
Cv = TfidfVectorizer(max_features=5000)
X = Cv.fit_transform(Train['text'])
Y = Train['target']

In [76]:
X = X.toarray()
Y = Y.values

In [77]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=0)

In [78]:
# Train and evaluate a Naïve Bayes classifier
Clf1 = MultinomialNB()
Clf1.fit(X_train, Y_train)
Pred = Clf1.predict(X_test)
Score_nb = metrics.accuracy_score(Y_test, Pred)

In [79]:
# Train and evaluate a Random Forest classifier
Clf2 = RandomForestClassifier(n_estimators=150)
Clf2.fit(X_train, Y_train)
Pred = Clf2.predict(X_test)
Score_rf = metrics.accuracy_score(Y_test, Pred)

In [80]:
# Train and evaluate a Decision Tree classifier
Clf3 = DecisionTreeClassifier(max_depth=100)
Clf3.fit(X_train, Y_train)
Pred = Clf3.predict(X_test)
Score_dt = metrics.accuracy_score(Y_test, Pred)

In [84]:
# Train and evaluate a Logistic Regression classifier
Clf4 = LogisticRegression(random_state=0)
Clf4.fit(X_train, Y_train)
Pred = Clf4.predict(X_test)
Accuracy = metrics.accuracy_score(Y_test, Pred)
Recall = metrics.recall_score(Y_test, Pred)
F1 = metrics.f1_score(Y_test, Pred)
print("Accuracy Score (Naive Bayes):", Score_nb)
print("Accuracy Score (Random Forest):", Score_rf)
print("Accuracy Score (Decision Tree):", Score_dt)
print("Accuracy Score (Logistic Regression):", Accuracy)
print("Recall Score (Logistic Regression):", Recall)
print("F1 Score (Logistic Regression):", F1)

Accuracy Score (Naive Bayes): 0.7974532431356944
Accuracy Score (Random Forest): 0.7958615200955034
Accuracy Score (Decision Tree): 0.759649820931158
Accuracy Score (Logistic Regression): 0.8058097890966972
Recall Score (Logistic Regression): 0.6803827751196172
F1 Score (Logistic Regression): 0.7445026178010471


In [88]:
# Train and evaluate an SVM classifier
Clf_svm = SVC(random_state=0)
Clf_svm.fit(X_train, Y_train)
Pred_svm = Clf_svm.predict(X_test)
Score_svm = metrics.accuracy_score(Y_test, Pred_svm)
print("Accuracy Score (SVM):", Score_svm)


Accuracy Score (SVM): 0.8010346199761241


In [91]:
Clf_xgb = xgb.XGBClassifier(random_state=0)
Clf_xgb.fit(X_train, Y_train)
Pred_xgb = Clf_xgb.predict(X_test)
Score_xgb = metrics.accuracy_score(Y_test, Pred_xgb)
print("Accuracy Score (XGBoost):", Score_xgb)

Accuracy Score (XGBoost): 0.7680063668921607
